# Dependencies

In [33]:
%pip install --quiet dotenv

Note: you may need to restart the kernel to use updated packages.


# Create an agent and thread

In [34]:
import os, time
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import ListSortOrder, MessageTextContent, MessageRole
from dotenv import load_dotenv
load_dotenv()

True

## Create an agent

In [35]:
agents_client = AgentsClient(
    endpoint=os.environ["AZURE_AI_FOUNDRY_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

In [36]:
# [START create_agent]
agent = agents_client.create_agent(
    model=os.environ["AZURE_AI_FOUNDRY_MODEL_DEPLOYMENT_NAME"],
    name="my-agent",
    instructions="You are helpful agent",
)

# [END create_agent]
print(f"Created agent, agent ID: {agent.id}")


Created agent, agent ID: asst_Oe481Ck4KQNTpJoPfHDuMz3B


## Create an agent

In [37]:

# [START create_thread]
thread = agents_client.threads.create()
# [END create_thread]
print(f"Created thread, thread ID: {thread.id}")

# [START create_message]
message = agents_client.messages.create(thread_id=thread.id, role="user", content="Hello, tell me a joke")
# [END create_message]
print(f"Created message, message ID: {message.id}")


Created thread, thread ID: thread_8aPzges9b52kXYrTMHZfz1dq
Created message, message ID: msg_jQm6bQpEZ9FoMvnPwwaUuIW0


## Create a message and process responses

In [38]:
# [START create_run]
run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id)

# Poll the run as long as run status is queued or in progress
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for a second
    time.sleep(1)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
    # [END create_run]
    print(f"Run status: {run.status}")

# [START list_messages]
messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

# The messages are following in the reverse order,
# we will iterate them and output only text contents.
for message in messages:
    last_message_content = message.content[-1]
    if isinstance(last_message_content, MessageTextContent):
        print(f"{message.role}: {last_message_content.text.value}")

for message in messages:
    if message.text_messages:
        last_text = message.text_messages[-1]
        print(f"{message.role}: {last_text.text.value}")

# Fetch and log all messages
messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for message in messages:
    if message.role == MessageRole.AGENT and message.url_citation_annotations:
        placeholder_annotations = {
            annotation.text: f" [see {annotation.url_citation.title}] ({annotation.url_citation.url})"
            for annotation in message.url_citation_annotations
        }
        for message_text in message.text_messages:
            message_str = message_text.text.value
            for k, v in placeholder_annotations.items():
                message_str = message_str.replace(k, v)
            print(f"{message.role}: {message_str}")
    else:
        for message_text in message.text_messages:
            print(f"{message.role}: {message_text.text.value}")

# [END list_messages]

Run status: in_progress
Run status: in_progress
Run status: completed
user: Hello, tell me a joke
assistant: Sure! Here's a joke for you:

Why did the scarecrow win an award?

Because he was outstanding in his field! 🌾😄
user: Hello, tell me a joke
assistant: Sure! Here's a joke for you:

Why did the scarecrow win an award?

Because he was outstanding in his field! 🌾😄


# Add tools

## Add Knowledge Tool: BingGroundingTool

In [39]:
from azure.ai.agents.models import MessageRole, BingGroundingTool
bing_conn_id = os.environ["BING_CONNECTION_ID"]
bing_conn_id2 = os.environ["BING_CONNECTION_ID2"]  

bing = BingGroundingTool(connection_id=bing_conn_id2, freshness="day", count=5, set_lang="en", market="us")

>Note: If you add a tool to agent.tools in your code after the agent has already been created and registered with the service, this change only affects your local Python object—it does not update the agent’s definition in the Azure service. The portal only shows tools that were present when the agent was created or last updated via the SDK or portal.

>Note: To have the portal reflect the new tool, you must update the agent’s definition in the service (for example, using an update or patch method if supported by the SDK, or by recreating the agent with the new tools list). Otherwise, local changes to agent.tools are not synchronized with the portal.


In [40]:
agent = agents_client.get_agent(agent.id)

#agent.tools.append(bing)
# Print agent details
print("Name:", agent.name)
print("Instructions:", agent.instructions)
print("Tools:", agent.tools)
print("Other metadata:", agent)

Name: my-agent
Instructions: You are helpful agent
Tools: []
Other metadata: {'id': 'asst_Oe481Ck4KQNTpJoPfHDuMz3B', 'object': 'assistant', 'created_at': 1750951366, 'name': 'my-agent', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are helpful agent', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}


In [41]:
new_tools = agent.tools + bing.definitions  # definitions is usually a list

agents_client.update_agent(
    agent_id=agent.id,
    model=agent.model,
    name=agent.name,
    instructions=agent.instructions,
    tools=new_tools,
    # add other required fields
)
agent = agents_client.get_agent(agent.id)
print(agent.tools)

[{'type': 'bing_grounding', 'bing_grounding': {'search_configurations': [{'connection_id': '/subscriptions/63855818-8b46-4d28-beb3-77974924d47e/resourceGroups/rg-basic-agent-setup/providers/Microsoft.CognitiveServices/accounts/aifoundysssb/projects/proj-default/connections/bingcustomsearch', 'market': 'us', 'set_lang': 'en', 'count': 5, 'freshness': 'day'}]}}]


In [42]:
# Create message to thread
message = agents_client.messages.create(
    thread_id=thread.id,
    role=MessageRole.USER,
    content="What is the latest news of Microsoft? Give me top 3 news articles with their summaries. Importantly, do analyze business impacts or implications of these news articles, show me only significant news articles that have business impacts.",
)
print(f"Created message, ID: {message.id}")

# Create and process agent run in thread with tools
run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Fetch run steps to get the details of the agent run
run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)
for step in run_steps:
    print(f"Step {step['id']} status: {step['status']}")
    step_details = step.get("step_details", {})
    tool_calls = step_details.get("tool_calls", [])
    
    if tool_calls:
        print("  Tool calls:")
        for call in tool_calls:
            print(f"    Tool Call ID: {call.get('id')}")
            print(f"    Type: {call.get('type')}")

            bing_grounding_details = call.get("bing_grounding", {})
            if bing_grounding_details:
                print(f"    Bing Grounding ID: {bing_grounding_details.get('requesturl')}")
    else:
        print("  No tool calls found in this step.")
        print(step_details)
    print()  # add an extra newline between steps

# Delete the agent when done
#agents_client.delete_agent(agent.id)
print("Deleted agent")

# Print the Agent's response message with optional citation
response_message = agents_client.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
if response_message:
    for text_message in response_message.text_messages:
        print(f"Agent response: {text_message.text.value}")
    for annotation in response_message.url_citation_annotations:
        print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")

Created message, ID: msg_ozkY4URRNgTd1BtkDA8jFLe5
Run finished with status: failed
Run failed: {'code': 'tool_user_error', 'message': 'Error: bing_search_user_error; [bing_search] Failed to call Get Custom Search Instance with status 401: { "statusCode": 401, "message": "Unauthorized. Access token is missing, invalid, audience is incorrect (https://bing.azure.com), or have expired." }'}
Step step_ZXEaCtab6G9sLgwJDWD6LuPF status: failed
  Tool calls:
    Tool Call ID: call_XLG7UtWRnWcokHolhFWvmeQ2
    Type: bing_grounding
    Bing Grounding ID: https://api.bing.microsoft.com/v7.0/search?q=latest Microsoft business news June 2025

Deleted agent
Agent response: Sure! Here's a joke for you:

Why did the scarecrow win an award?

Because he was outstanding in his field! 🌾😄


## Add Action tool: A Logic App tool

It seems at moment, a custom made python util [functions/classes](https://github.com/azure-ai-foundry/foundry-samples/blob/a1b346f0085126da2bd13ae927d5e83fd7e18fb2/samples/microsoft/python/getting-started-agents/logic_apps/user_logic_apps.py) to register and invoke Azure Logic Apps. The script is downloaded and stored under *utils* folder in this repo.

>Note: At moment consumption based logic app is supported (Not standard logic app). Does the consumption based logic app support private endpoints?

In [ ]:
import os
import sys
from typing import Set

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import ToolSet, FunctionTool
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
load_dotenv()



# Import AzureLogicAppTool and the function factory from user_logic_apps
from utils.logicapp import AzureLogicAppTool, create_send_email_function

# [START register_logic_app]
# Logic App details
# Extract subscription and resource group from the project scope
subscription_id = os.environ["AZURE_SUBSCRIPTION_ID"]
resource_group = os.environ["AZURE_AI_FOUNDRY_RESOURCE_GROUP"]
logic_app_name = os.environ["AZURE_LOGIC_APP_NAME"]
trigger_name = os.environ["AZURE_LOGIC_APP_TRIGGER_NAME"]
bing_conn_id2 = os.environ["BING_CONNECTION_ID2"]  


# Create and initialize AzureLogicAppTool utility
logic_app_tool = AzureLogicAppTool(subscription_id, resource_group)
logic_app_tool.register_logic_app(logic_app_name, trigger_name)
print(f"Registered logic app '{logic_app_name}' with trigger '{trigger_name}'.")

# Create the specialized "send_email_via_logic_app" function for your agent tools
send_email_func = create_send_email_function(logic_app_tool, logic_app_name)

# Prepare the function tools for the agent
functions_to_use: Set = {
    #fetch_current_datetime,
    send_email_func,  # This references the AzureLogicAppTool instance via closure
}
# [END register_logic_app]

# Create an agent
functions = FunctionTool(functions=functions_to_use)
#bing2 = BingGroundingTool(connection_id=bing_conn_id2, freshness="day", count=5, set_lang="en", market="us")


toolset = ToolSet()
toolset.add(functions)
#toolset.add(bing2)

print(toolset.definitions)
#agents_client.enable_auto_function_calls(toolset)

agent = agents_client.get_agent(agent.id)

agent = agents_client.get_agent("asst_NppJTDYBhBMkpYpGhWjKLppa")


#agent.tools.append(bing)
# Print agent details
print("Name:", agent.name)
print("Instructions:", agent.instructions)
print("Tools:", agent.tools)
print("Other metadata:", agent)


Registered logic app 'Send-email' with trigger 'When_a_HTTP_request_is_received'.
[{'type': 'function', 'function': {'name': 'send_email_via_logic_app', 'description': 'Sends an email by invoking the specified Logic App with the given recipient, subject, and body.', 'parameters': {'type': 'object', 'properties': {'recipient': {'type': 'string', 'description': 'The email address of the recipient.'}, 'subject': {'type': 'string', 'description': 'The subject of the email.'}, 'body': {'type': 'string', 'description': 'The body of the email.'}}, 'required': ['recipient', 'subject', 'body']}}}]
Name: my-agent
Instructions: You are helpful agent
Tools: [{'type': 'bing_grounding', 'bing_grounding': {'search_configurations': [{'connection_id': '/subscriptions/63855818-8b46-4d28-beb3-77974924d47e/resourceGroups/rg-basic-agent-setup/providers/Microsoft.CognitiveServices/accounts/aifoundysssb/projects/proj-default/connections/bingcustomsearch', 'market': 'us', 'set_lang': 'en', 'count': 5, 'freshn

In [44]:
bing_conn_id2

'/subscriptions/63855818-8b46-4d28-beb3-77974924d47e/resourceGroups/rg-basic-agent-setup/providers/Microsoft.CognitiveServices/accounts/aifoundysssb/projects/proj-default/connections/bingcustomsearch'

In [45]:
bing.definitions

[{'type': 'bing_grounding', 'bing_grounding': {'search_configurations': [{'connection_id': '/subscriptions/63855818-8b46-4d28-beb3-77974924d47e/resourceGroups/rg-basic-agent-setup/providers/Microsoft.CognitiveServices/accounts/aifoundysssb/projects/proj-default/connections/bingcustomsearch', 'market': 'us', 'set_lang': 'en', 'count': 5, 'freshness': 'day'}]}}]

In [14]:
toolset.definitions

[{'type': 'function', 'function': {'name': 'send_email_via_logic_app', 'description': 'Sends an email by invoking the specified Logic App with the given recipient, subject, and body.', 'parameters': {'type': 'object', 'properties': {'recipient': {'type': 'string', 'description': 'The email address of the recipient.'}, 'subject': {'type': 'string', 'description': 'The subject of the email.'}, 'body': {'type': 'string', 'description': 'The body of the email.'}}, 'required': ['recipient', 'subject', 'body']}}},
 {'type': 'bing_grounding', 'bing_grounding': {'search_configurations': [{'connection_id': '/subscriptions/63855818-8b46-4d28-beb3-77974924d47e/resourceGroups/rg-basic-agent-setup/providers/Microsoft.CognitiveServices/accounts/aifoundysssb/projects/proj-default/connections/bingcustomsearch', 'market': 'us', 'set_lang': 'en', 'count': 5, 'freshness': 'day'}]}}]

In [46]:
from pprint import pprint
import json

# Merge toolset.definitions with agent.tools, avoiding duplicate tool names
def get_tool_name(tool):
    if tool.get("type") == "function":
        return tool["function"]["name"]
    if tool.get("type") == "bing_grounding":
        configs = tool.get("bing_grounding", {}).get("search_configurations", [])
        if configs and "connection_id" in configs[0]:
            return configs[0]["connection_id"].rstrip("/").split("/")[-1]
        return "bing_grounding"

existing_names = {get_tool_name(t) for t in agent.tools}
print(f"Existing tool names: {existing_names}")
new_tool_names = {get_tool_name(t) for t in toolset.definitions}
print(f"New tool names: {new_tool_names}")
new_tools = agent.tools + [t for t in toolset.definitions if get_tool_name(t) not in existing_names]
pprint(agent.tools, indent=2)


agents_client.update_agent(
    agent_id=agent.id,
    model=agent.model,
    name=agent.name,
    instructions=agent.instructions,
    tools=new_tools,
    # add other required fields
)
agent = agents_client.get_agent(agent.id)
pprint(agent.tools, indent=2)
# Print agent details
print("Name:", agent.name)
print("Instructions:", agent.instructions)
print("Tools:", agent.tools)
print("Other metadata:", agent)

Existing tool names: {'bingcustomsearch'}
New tool names: {'send_email_via_logic_app'}
[ {'type': 'bing_grounding', 'bing_grounding': {'search_configurations': [{'connection_id': '/subscriptions/63855818-8b46-4d28-beb3-77974924d47e/resourceGroups/rg-basic-agent-setup/providers/Microsoft.CognitiveServices/accounts/aifoundysssb/projects/proj-default/connections/bingcustomsearch', 'market': 'us', 'set_lang': 'en', 'count': 5, 'freshness': 'day'}]}}]
[ {'type': 'bing_grounding', 'bing_grounding': {'search_configurations': [{'connection_id': '/subscriptions/63855818-8b46-4d28-beb3-77974924d47e/resourceGroups/rg-basic-agent-setup/providers/Microsoft.CognitiveServices/accounts/aifoundysssb/projects/proj-default/connections/bingcustomsearch', 'market': 'us', 'set_lang': 'en', 'count': 5, 'freshness': 'day'}]}},
  {'type': 'function', 'function': {'name': 'send_email_via_logic_app', 'description': 'Sends an email by invoking the specified Logic App with the given recipient, subject, and body.',

In [16]:
# Print agent details
print("Name:", agent.name)
print("Instructions:", agent.instructions)
print("Tools:", agent.tools)
print("Other metadata:", agent)

Name: my-agent
Instructions: You are helpful agent
Tools: [{'type': 'bing_grounding', 'bing_grounding': {'search_configurations': [{'connection_id': '/subscriptions/63855818-8b46-4d28-beb3-77974924d47e/resourceGroups/rg-basic-agent-setup/providers/Microsoft.CognitiveServices/accounts/aifoundysssb/projects/proj-default/connections/bingsearch', 'market': 'us', 'set_lang': 'en', 'count': 5, 'freshness': 'day'}]}}, {'type': 'function', 'function': {'name': 'send_email_via_logic_app', 'description': 'Sends an email by invoking the specified Logic App with the given recipient, subject, and body.', 'parameters': {'type': 'object', 'properties': {'recipient': {'type': 'string', 'description': 'The email address of the recipient.'}, 'subject': {'type': 'string', 'description': 'The subject of the email.'}, 'body': {'type': 'string', 'description': 'The body of the email.'}}, 'required': ['recipient', 'subject', 'body']}, 'strict': False}}, {'type': 'bing_grounding', 'bing_grounding': {'search_c

In [ ]:
from pprint import pprint
pprint(agent.tools, indent=2)

[ {'type': 'function', 'function': {'name': 'send_email_via_logic_app', 'description': 'Sends an email by invoking the specified Logic App with the given recipient, subject, and body.', 'parameters': {'type': 'object', 'properties': {'recipient': {'type': 'string', 'description': 'The email address of the recipient.'}, 'subject': {'type': 'string', 'description': 'The subject of the email.'}, 'body': {'type': 'string', 'description': 'The body of the email.'}}, 'required': ['recipient', 'subject', 'body']}, 'strict': False}},
  {'type': 'bing_grounding', 'bing_grounding': {'search_configurations': [{'connection_id': '/subscriptions/63855818-8b46-4d28-beb3-77974924d47e/resourceGroups/rg-basic-agent-setup/providers/Microsoft.CognitiveServices/accounts/aifoundysssb/projects/proj-default/connections/bingsearch', 'market': 'us', 'set_lang': 'en', 'count': 5, 'freshness': 'day'}]}}]


In [ ]:

print(f"Created agent, ID: {agent.id}")

# Create a thread for communication
thread = agents_client.threads.create()
print(f"Created thread, ID: {thread.id}")

# Create a message in the thread
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content="Hello, please send an email to chengyuliu@microsoft.com with greeting",
)
print(f"Created message, ID: {message.id}")

# Create and process an agent run in the thread
run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Delete the agent when done
#agents_client.delete_agent(agent.id)
print("Deleted agent")

# Fetch and log all messages
messages = agents_client.messages.list(thread_id=thread.id)
for msg in messages:
    if msg.text_messages:
        last_text = msg.text_messages[-1]
        print(f"{msg.role}: {last_text.text.value}")

# Clean up

In [ ]:
#agents_client.delete_agent(agent.id)
print("Deleted agent")
#agents_client.threads.delete(thread.id)
print("Deleted thread")

Deleted agent


Deleted thread
